# T5. REFERENCE LEVELS

In this notebook, we extract the reference levels from the t1_semantic segmentation/ t2_instance segmentation

## LIBRARIES

In [1]:
#IMPORT PACKAGES
from rdflib import Graph, URIRef
import os.path
import importlib
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import multiprocessing
import random as rd
import pandas as pd
# from tabulate import tabulate
import cv2
import laspy
import json
from scipy.spatial.transform import Rotation   
import copy
import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl
import geomapi.tools.progresstools as pt

#import utils
import context 
import utils as utl


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

## INPUTS

In [4]:
#paths
path=Path(os.getcwd()).parents[0] # with MB this is 2

print(path)
input_folder=path/'data'/'t4'/'test' 
class_file=path/'data'/'_classes.json'
output_folder=path/'data'/'t5'/ 'test'
os.makedirs(output_folder, exist_ok=True)


/home/sdegeyter/Code/Scan-to-BIM-CVPR-2024


Import Classes

In [6]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

{'classes': [{'name': 'Unassigned', 'id': 255, 'temp_id': 0, 'color': '#9da2ab'}, {'name': 'Floors', 'id': 0, 'temp_id': 1, 'color': '#03c2fc'}, {'name': 'Ceilings', 'id': 1, 'temp_id': 2, 'color': '#e81416'}, {'name': 'Walls', 'id': 2, 'temp_id': 3, 'color': '#ffa500'}, {'name': 'Columns', 'id': 3, 'temp_id': 4, 'color': '#faeb36'}, {'name': 'Doors', 'id': 4, 'temp_id': 5, 'color': '#79c314'}, {'name': 'Windows', 'id': 5, 'temp_id': 6, 'color': '#4b369d'}], 'default': 255, 'type': 'semantic_segmentation', 'format': 'kitti', 'created_with': {'name': 'Saiga', 'version': '1.0.1'}}


Import PCD

In [17]:
files=utl.get_list_of_files(input_folder,'.laz')

for f in files[:1]: #only read the first one
    pcdNodes=[]
    
    # check if las/pcd variable is already defined    
    print(f'processing {ut.get_filename(f)}...')      
    las = laspy.read(f) if 'las' not in globals() else las
    pcd=gmu.las_to_pcd(las,getNormals=True) if 'pcd' not in globals() else pcd # this is the slowest step
        
    #seperate initial objects
    for c in class_dict['classes']:
        print(c['name'])
        idx=np.where((las['classes']==c['id']))[0]
        class_pcd=pcd.select_by_index(idx)
        object_labels=las['objects'][idx]
        
        for j in np.unique(object_labels):
            indices=np.where(object_labels==j)[0]
            object_pcd=class_pcd.select_by_index(indices)
            pcdNodes.append(PointCloudNode(resource=object_pcd,
                                        class_id=c['id'],
                                        object_id=j,
                                        color=ut.random_color(),
                                        name=c['name']+f'_{str(j)}'))
            
            
    #all further processing will be placed here (or in functions)!
    print( c['name'], f': {len(pcdNodes)} Nodes found')             

processing 05_MedOffice_01_F2_small1...
Unassigned
(10399439,)
Floors
(2867767,)
Ceilings
(5737921,)
Walls
(2633,)
(75779,)
(6000,)
(12508,)
(1463,)
(1783,)
(1615,)
(12447,)
(7897,)
(13767,)
(13068,)
(17022,)
(26220,)
(136808,)
(38772,)
(17285,)
(91748,)
(40086,)
(5704,)
(25575,)
(15473,)
(10951,)
(121004,)
(18036,)
(25734,)
(53866,)
(27951,)
(45054,)
(24839,)
(62241,)
(33995,)
(20824,)
(147797,)
(40292,)
(48208,)
(18537,)
(85906,)
(18753,)
(78267,)
(47624,)
(22233,)
(23384,)
(92197,)
(15005,)
(86209,)
(39493,)
(23639,)
(17172,)
(7914,)
(77504,)
(178555,)
(57908,)
(82997,)
(55895,)
(36962,)
(25456,)
(48337,)
(29462,)
(27529,)
(94811,)
(55502,)
(5562,)
(4597,)
(17388,)
(7427,)
(9134,)
(58566,)
(5950,)
(13893,)
(192041,)
(77852,)
(14704,)
(51669,)
(135990,)
(152219,)
(108725,)
(104598,)
(65029,)
(36705,)
(9691,)
(17264,)
(39017,)
(45576,)
(48533,)
(60233,)
(35499,)
(123340,)
(10070,)
(6274,)
(20929,)
(34192,)
(63152,)
(26725,)
(6717,)
(35798,)
(74614,)
(39927,)
(29904,)
(17349,)
(39013,)

In [ ]:
# joined_pcd=gmu.join_geometries([p.resource.paint_uniform_color(ut.random_color()) for p in objectNodes])
# o3d.visualization.draw_geometries([joined_pcd,gmu.sample_geometry(class_pcd)[0]])

import Graph

In [18]:
# graph=Graph().parse(graphPath)
# pcdNodes=tl.graph_to_nodes(graph)

match point clouds with graph

In [19]:
# for n in pcdNodes: # this is 6 times slower than the code above !!
#     idx=np.where((laz['classes']==n.class_id) & (laz['objects']==n.object_id))
#     pcd=o3d.geometry.PointCloud()
#     pcd.points=o3d.utility.Vector3dVector(laz.xyz[idx])
#     n.resource=pcd

In [20]:
# {key:value for key, value in pcdNodes[1].__dict__.items() if not key.startswith('__') and not callable(key)}              

In [133]:
# joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(ut.literal_to_array(n.color)) for n in pcdNodes if n.resource is not None])
# o3d.visualization.draw_geometries([joined_pcd])

## PROCESSING

get floor point clouds and upper ceiling node

In [26]:
#retrieve floor points
floorNodes=[n for n in pcdNodes if n.class_id ==0]
ceilingNodes=[n for n in pcdNodes if n.class_id ==1]
print(f'{len(floorNodes)} floorNodes detected!')
for n in floorNodes:
    print(f' {n.name} with {n.get_oriented_bounding_box()}')


1 floorNodes detected!
 Floors_1 with OrientedBoundingBox: center: (7.15942, 9.13511, -1.74057), extent: 88.7838, 51.6765, 5.28601)


In [28]:
# for now, put all floorNodes in a single group (1 reference level)
groupedLevelNodes=[floorNodes,ceilingNodes]
print(groupedLevelNodes)


[[<geomapi.nodes.pointcloudnode.PointCloudNode object at 0x7fb428356980>], [<geomapi.nodes.pointcloudnode.PointCloudNode object at 0x7fb428355b10>]]


Compute reference Planes

In [29]:
#compute height underneight all floor points of a level (according to the lowest slab)
referenceNodes= []

for i,nodes in enumerate(groupedLevelNodes):
    #create sessionNode
    name='level_'+str(i)+'0'
    referenceNode=SessionNode(linkedNodes=nodes,
                                name=name,
                                subject=name)
    
    #determine height -> note that this can be negative
    weights=[float(len(np.asarray(n.resource.points))) for n in nodes]
    heights= [float(n.cartesianTransform[2,3]) for n in nodes]
    weighted_height= np.average(heights, weights=weights)

    #compute plane from cornerpoints orientedbounding box
    vertices=np.array([np.hstack((referenceNode.orientedBounds[0][0:2],weighted_height)),
                       np.hstack((referenceNode.orientedBounds[1][0:2],weighted_height)),
                       np.hstack((referenceNode.orientedBounds[2][0:2],weighted_height)),
                       np.hstack((referenceNode.orientedBounds[4][0:2],weighted_height))])#,
    vertices=o3d.utility.Vector3dVector(vertices)
    triangles=o3d.utility.Vector3iVector(np.array([[0,1,2],[2,1,3]]))
    plane=o3d.geometry.TriangleMesh(vertices,triangles)

    #assign information to referenceNode
    referenceNode.plane=plane
    referenceNode.height=weighted_height
    
    referenceNodes.append(referenceNode)
    print(f' Level {referenceNode.name} created at height {referenceNode.height}')
# print(f'{len(referenceNodes)} referenceNodes created!')

 Level level_00 created at height -0.007221328826867619
 Level level_10 created at height 2.7739179168203005


In [138]:
# joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(n.color) for n in pcdNodes if n.resource is not None])
# o3d.visualization.draw_geometries([joined_pcd,referenceNodes[0].plane])

In [30]:
{key:value for key, value in referenceNode.__dict__.items() if not key.startswith('__') and not callable(key)}              

{'_linkedNodes': [<geomapi.nodes.pointcloudnode.PointCloudNode at 0x7fb428355b10>],
 '_linkedSubjects': [rdflib.term.URIRef('file:///Ceilings_2')],
 '_subject': rdflib.term.URIRef('file:///level_10'),
 '_graph': None,
 '_graphPath': None,
 '_path': None,
 '_name': 'level_10',
 '_cartesianBounds': array([-2.80895652e+01,  3.97142580e+01, -3.60133539e+01,  5.81279315e+01,
        -4.41641246e-02,  7.53458671e+00]),
 '_orientedBounds': array([[ 2.14217984e+01, -3.58783279e+01, -4.41641246e-02],
        [ 3.96821373e+01,  4.72735862e+01,  2.26753106e+00],
        [-2.80895652e+01, -2.50239826e+01,  6.19715839e-01],
        [ 2.14539191e+01, -3.60133539e+01,  4.55901155e+00],
        [-9.79710564e+00,  5.79929055e+01,  7.53458671e+00],
        [-2.80574445e+01, -2.51590086e+01,  5.22289152e+00],
        [ 3.97142580e+01,  4.71385602e+01,  6.87070674e+00],
        [-9.82922629e+00,  5.81279315e+01,  2.93141103e+00]]),
 '_orientedBoundingBox': OrientedBoundingBox: center: (5.81235, 11.0573, 3

Compute Json information

In [32]:
for n in referenceNodes:
    n.box=n.resource.get_oriented_bounding_box()
    n.box.color=[1,0,0]
    
    rotation_matrix=copy.deepcopy(n.box.R) #! can these angles be negative?
    r =  Rotation.from_matrix(np.asarray(rotation_matrix))
    n.rotations = r.as_euler("zyx",degrees=True)

## EXPORT

geometry

In [33]:
joined_references=gmu.join_geometries([n.resource for n in referenceNodes])
success=o3d.io.write_triangle_mesh(filename=os.path.join(output_folder,f'{ut.get_filename(f)}_levels.obj'), mesh=joined_references) 
print(f' Saving joint references : {success}')
    

 Saving joint references : True


json with reference heights

In [34]:
#declare json
json_data = {
        "filename": f'{ut.get_filename(f)}_levels.obj',
        "objects": []
    }

In [35]:
#fill json
for n in referenceNodes:
    obj = {
            "name": 'level_'+n.name,
            "centroid": {
                "x": n.box.center[0],
                "y": n.box.center[1],
                "z": n.height
            },
            "dimensions": {
                "length": n.box.extent[0],
                "width": n.box.extent[1],
                "height": n.height
            },
            "rotations": {
                "x": 0,
                "y": 0,
                "z": n.rotations[0]
            }
            }
    json_data["objects"].append(obj)

In [36]:
#write this information to the 3D detection json
with open(os.path.join(output_folder,f'{ut.get_filename(f)}_levels.json') , "w") as json_file: 
    json.dump(json_data, json_file, indent=4)
print("JSON data written to file:", os.path.join(output_folder,f'{ut.get_filename(f)}_levels.json') )

JSON data written to file: /home/sdegeyter/Code/Scan-to-BIM-CVPR-2024/data/t5/test/05_MedOffice_01_F2_small1_levels.json


save Graph

In [40]:
graphPath=os.path.join(output_folder,f'{ut.get_filename(f)}_levels.ttl')
graph=tl.nodes_to_graph(referenceNodes,
                  graphPath=graphPath,save=True)


In [41]:
print(graph.serialize())

@prefix e57: <http://libe57.org#> .
@prefix openlabel: <https://www.asam.net/index.php?eID=dumpFile&t=f&f=3876&token=413e8c85031ae64cc35cf42d0768627514868b2f#> .
@prefix v4d: <https://w3id.org/v4d/core#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<file:///level_00> a v4d:SessionNode ;
    e57:cartesianBounds """[-30.50617846  44.82502113 -40.88119715  59.15142274  -5.77151495
   2.29038091]""" ;
    e57:cartesianTransform """[[ 1.          0.          0.          7.15942134]
 [ 0.          1.          0.          9.1351128 ]
 [ 0.          0.          1.         -1.74056702]
 [ 0.          0.          0.          1.        ]]""" ;
    v4d:box "OrientedBoundingBox: center: (7.15942, 9.13511, -1.74057), extent: 88.7838, 51.6765, 5.28601)" ;
    v4d:height "-0.007221328826867619"^^xsd:float ;
    v4d:linkedSubjects "['file:///Floors_1']" ;
    v4d:name "level_00" ;
    v4d:orientedBounds """[[ 18.93030764 -40.88119715  -3.9889792 ]
 [ 44.69676502  44.06274756  -5.77151495]
 [-3